In [ ]:
import numpy as np
import torch
from torch import nn

In [ ]:
# Data generation
n = 100 #number of samples
np.random.seed(0)
X_tmp = np.random.randn(n,3)
X = np.zeros(shape=(n,5))
X[:,0] = X_tmp[:,0]
X[:,2] = X_tmp[:,1]
X[:,4] = X_tmp[:,2]

print(X[:4])

In [ ]:
X = X - np.mean(X, axis=0)  # mean centering

print(X[:4])

In [ ]:
X_tensor = torch.tensor(X, dtype=torch.float32).unsqueeze(1)

In [ ]:
X_tensor[:4]

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.act = nn.Tanh()

        self.enc1 = nn.Linear(5, 4)
        self.enc2 = nn.Linear(4, 3)

        self.dec1 = nn.Linear(3, 4)
        self.dec2 = nn.Linear(4, 5)

    def E(self, x):
        x = self.act(self.enc1(x))
        x = self.act(self.enc2(x))
        return x

    def D(self, x):
        x = self.act(self.dec1(x))
        x = self.dec2(x)
        return x

    def forward(self, x):
        z = self.E(x)
        return self.D(z)

In [ ]:
lr = 0.001
epochs = 10000

In [ ]:
model = AutoEncoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
model(X_tensor)[:4]

In [ ]:
# Training
for epoch in range(epochs):
    optimizer.zero_grad()
    x_hat = model(X_tensor)
    loss = criterion(x_hat, X_tensor)
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item() / n:.6f}")

In [ ]:
z = model.E(X_tensor)

In [ ]:
print(z[:4])

In [ ]:
X_hat = model.D(z).detach().numpy()

In [ ]:
print(np.round(X_hat[:4],2))

In [ ]:
print(X[:4])

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
latent = z.detach().numpy().squeeze()

fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(latent[:, 0], latent[:, 1], latent[:, 2], alpha=0.7)
ax.set_title("Latent Space (3D)")
plt.show()